In [1]:
import geopandas as gpd
import networkx as nx
from shapely.geometry import Point
import json
from shapely import to_geojson
from shapely.geometry import shape
from networkx.readwrite import json_graph
import folium
from util import graph_to_geojson, graph_from_geojson

# Creation of Graphs from the geojson files obtained
data from - https://data.humdata.org/dataset/hotosm_svk_railways
          - https://data.humdata.org/dataset/hotosm_cze_railways

In [2]:
country_codes: list[str] = ["svk",  "cze"] 
geographical_centers: list[list[float]] = [
    [48.7, 19.5],            # Slovakia (svk)
    [49.75, 15.5]            # Czechia (cze)
]
for idx, country in enumerate(country_codes):
    print(country)
    print(geographical_centers[idx])

svk
[48.7, 19.5]
cze
[49.75, 15.5]


In [ ]:
for idx, country in enumerate(country_codes):
    # loading data
    stations_gdf =  gpd.read_file(f"../data/hotosm_{country}_railways_points_geojson.geojson")
    stations_gdf = stations_gdf[stations_gdf['railway'] == 'station']
    tracks_gdf = gpd.read_file(f"../data/hotosm_{country}_railways_lines_geojson.geojson")
    tracks_gdf = tracks_gdf[tracks_gdf['railway'] == 'rail'] 
    # graph creation
    G = nx.Graph()

    for index, row in stations_gdf.iterrows():
        if row["name"] and row["geometry"]:
            G.add_node(row["name"], geometry=row["geometry"], id=row["osm_id"])
        
    # Add edges based on tracks
    for idx, track in tracks_gdf.iterrows():
        start_point = Point(track.geometry.coords[0])
        end_point = Point(track.geometry.coords[-1])

        # Find nearest station to start
        start_station = stations_gdf.iloc[stations_gdf.geometry.distance(start_point).idxmin()]
        start_city = start_station['name']

        # Find nearest station to end
        end_station = stations_gdf.iloc[stations_gdf.geometry.distance(end_point).idxmin()]
        end_city = end_station['name']

        # Add edge if cities are different
        if start_city != end_city and start_city and end_city:  # Ensure cities exist
            G.add_edge(start_city, end_city)
    

    graph_to_geojson(G=G, path=f"../graphs/{country}-railroad-network_without_distance.json")
    try:
        m = folium.Map(location=geographical_centers[idx], zoom_start=7)

        for node1, node2 in G.edges():
            if node1 in G.nodes and node2 in G.nodes and 'geometry' in G.nodes[node1] and 'geometry' in G.nodes[node2]:
                geom1 = G.nodes[node1]['geometry']
                geom2 = G.nodes[node2]['geometry']
                if isinstance(geom1, Point) and isinstance(geom2, Point):
                    lat1, lon1 = geom1.y, geom1.x
                    lat2, lon2 = geom2.y, geom2.x
                    folium.PolyLine([[lat1, lon1], [lat2, lon2]], color='blue', weight=1).add_to(m)
                else:
                    print(f"Warning: Skipping edge between {node1} and {node2} due to non-Point geometry.")
            else:
                print(f"Warning: Skipping edge between {node1} and {node2} because one or both nodes are missing geometry information.")

        # Mark the nodes (stations) with red dots
        for node, data in G.nodes(data=True):
            if 'geometry' in data and isinstance(data['geometry'], Point):
                folium.CircleMarker(
                    location=[data['geometry'].y, data['geometry'].x],
                    radius=1,
                    color='red',
                    fill=True,
                    fill_color='red',
                    fill_opacity=0.7
                ).add_to(m)

        # Save or display the map
        m.save(f'visualizations/{country}_railroads.html')
    except:
        pass

C:\Users\pazera\AppData\Local\Temp\ipykernel_22832\906148296.py:31: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  start_station = stations_gdf.iloc[stations_gdf.geometry.distance(start_point).idxmin()]
C:\Users\pazera\AppData\Local\Temp\ipykernel_22832\906148296.py:35: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  end_station = stations_gdf.iloc[stations_gdf.geometry.distance(end_point).idxmin()]
C:\Users\pazera\AppData\Local\Temp\ipykernel_22832\906148296.py:31: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  start_station = stations_gdf.iloc[stations_gdf.geometry.distanc

In [ ]:
from geopy.distance import geodesic
country_codes: list[str] = ["cze", "svk"]  
for c_c in country_codes:
    G = graph_from_geojson(path=f"../graphs/{c_c}-railroad-network_without_distance.json")
    for u, v in G.edges():
        # Fallback to straight-line distance using node geometries
        point_u = G.nodes[u]['geometry']
        point_v = G.nodes[v]['geometry']
        coords_u = point_u.coords[0]
        coords_v = point_v.coords[0]
        latlon_u = (coords_u[1], coords_u[0])
        latlon_v = (coords_v[1], coords_v[0])
        total_distance = geodesic(latlon_u, latlon_v).meters
        
        G[u][v]['distance'] = total_distance        
    graph_to_geojson(G=G, path=f"../graphs/{c_c}-railroad-network_with_distance.json")

c:\Users\pazera\anaconda3\envs\starter\Lib\site-packages\networkx\readwrite\json_graph\node_link.py:287: FutureWarning: 
The default value will be changed to `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_graph(data, edges="links") to preserve current behavior, or
  nx.node_link_graph(data, edges="edges") for forward compatibility.
  warnings.warn(
c:\Users\pazera\anaconda3\envs\starter\Lib\site-packages\networkx\readwrite\json_graph\node_link.py:142: FutureWarning: 
The default value will be `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_data(G, edges="links") to preserve current behavior, or
  nx.node_link_data(G, edges="edges") for forward compatibility.
  warnings.warn(


# Manual fixing of the data for Slovakia

In [ ]:
G = graph_from_geojson(path=f"../graphs/svk-railroad-network_with_distance.json")


In [36]:
stations = list(G.nodes())
for station in stations:
    
    print(station)

Spišská Belá
Podhorany pri Kežmarku
Podolínec
Spišská Belá zastávka
Bušovce
Hronec
Devínska Nová Ves
Brezno
Malacky
Čaňa
Krásna nad Hornádom
Barca
Malé Straciny
Stará Ľubovňa
Hodejov
Červená Skala
Rimavská Seč
Blhovce
Süttő
Piszke
Balnica
Pohronský Ruskov
Likier
Pribeník
Budkovce ŠRT
Michaľany
Veľké Kapušany
Vojany ŠRT
Čierna nad Tisou
Rožňava
Plavnica
Kittsee
Kunova Teplica
Štítnik
Veľké Leváre
Sekule
Svit
Štrkovec
Ochtiná
Hnilec
Vlachovo
Dobšiná
Nižná Slaná
Muráň
Mlynky
Betliar
Gemerská Hôrka
Jelšava
Martin
Tanečník
Sedlo Beskyd
Čremošné
Radvaň
Počkaj
Kostiviarska
Harmanec jaskyňa
Jablonov nad Turňou
Spišské Podhradie
Hrhov
Moldava nad Bodvou
Švedlár
Pstruša
Slovenská Ľupča
Dobrá Niva
Vpred
Poltár
Banská Belá
Ipeľský Sokolec
Čata
Zohor
Dlhé nad Cirochou
Hontianske Nemce
Strážske
Hrinište
Kúty
Štúrovo
Štúrovo tranzitná skupina
Kysucké Nové Mesto
Žihárec
Rimavské Zalužany
Rimavská Baňa
Pribeta
Strekov
Krásno nad Kysucou
Dvory nad Žitavou
Košice
Čadca
Soľ
Drienovská Nová Ves
Gbely
Holíč

# Deleted stations Slovakia

In [37]:
from networkx import NetworkXError
stations_to_remove = ["Vydrovo - Koliba", "Balnica", "Starý Smokovec", "Hrebienok", "Štrbské Pleso OŽ", "Štrbské Pleso", "Ružomberok-Zápalkáreň",
                      "Ružomberok - Malé Nádražie","Chmúra", "Tanečník", "Sedlo Beskyd", "Bratislava - Hlavná stanica"]
for node in stations_to_remove:
    try:
        G.remove_node(node)
    except NetworkXError:
        pass

# Added stations Slovakia

In [38]:
G.add_node("Bratislava - Hlavná stanica", geometry=Point(17.10416625, 48.15499938))
G.add_node("Bratislava - Vinohrady", geometry=Point(17.13387381009982, 48.18698557873771))

# Added connections Slovakia


In [5]:
from geopy.distance import geodesic

In [39]:

edges_list = [("Malé Straciny", "Lučenec"), ("Hronec zlievareň", "Čierny Balog"), ("Petrovce nad Laborcom", "Michalovce"),
              ("Vpred", "Čermeľ"), ("Alpinka", "Vpred"), ("Bratislava - Hlavná stanica", "Bratislava - Vinohrady"), 
              ("Bratislava - Vinohrady", "Sládkovičovo"), ("Bratislava-Lamač", "Bratislava - Hlavná stanica")]
for u, v in edges_list:
    G.add_edge(u, v)
    point_u = G.nodes[u]['geometry']
    point_v = G.nodes[v]['geometry']
    coords_u = point_u.coords[0]
    coords_v = point_v.coords[0]
    latlon_u = (coords_u[1], coords_u[0])
    latlon_v = (coords_v[1], coords_v[0])
    total_distance = geodesic(latlon_u, latlon_v).meters
    G[u][v]['distance'] = total_distance        



In [6]:
edges_list = [("Orechová Potôň", "Dunajská Streda"), ("Čečejovce", "Veľká Ida")]
for u, v in edges_list:
    G.add_edge(u, v)
    point_u = G.nodes[u]['geometry']
    point_v = G.nodes[v]['geometry']
    coords_u = point_u.coords[0]
    coords_v = point_v.coords[0]
    latlon_u = (coords_u[1], coords_u[0])
    latlon_v = (coords_v[1], coords_v[0])
    total_distance = geodesic(latlon_u, latlon_v).meters
    G[u][v]['distance'] = total_distance        

# Saving the fixed graph


In [ ]:
graph_to_geojson(G=G, path="../graphs/svk-railroad-network.json" )

c:\Python312\Lib\site-packages\networkx\readwrite\json_graph\node_link.py:142: FutureWarning: 
The default value will be `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_data(G, edges="links") to preserve current behavior, or
  nx.node_link_data(G, edges="edges") for forward compatibility.
  warnings.warn(


# Manual fixing Czechia

In [ ]:
G = graph_from_geojson(path=f"../graphs/cze-railroad-network_with_distance.json")
stations = list(G.nodes())
for station in stations:
    print(station)

Zliv
Hluboká nad Vltavou
Božejovice
Branice
Český Krumlov
Náchod
Křemže
Temelín
Týn nad Vltavou
Plzeň-Křimice
Hýskov
Staňkov
Lužany
Liběšice
Skuteč
Františkov nad Ploučnicí
Polepy
Libčice nad Vltavou
Kynšperk nad Ohří
Hřebeny
Oloví
Svatava zastávka
Kraslice
Potůčky
Pernink
Planá u Mariánských Lázní
Kraslice předměstí
Luby u Chebu
Nová Role
Kateřina
Svatava
Nové Hamry
Valy u Mariánských Lázní
Soos
Běšiny
Nejdek
Ústí nad Labem hlavní nádraží
Rychnov u Jablonce nad Nisou
Kolinec
Kadaň-Prunéřov
Vejprty
Bohušovice nad Ohří
Litvínov
Třebenice
Úpořiny
Trmice
Mutějovice
Třebívlice
Žatec
Niederschlag
Mšené Lázně
Hostivice
Praha-Ruzyně
Dolejší Hůrky
Oberwiesenthal
Hammerunterwiesenthal
Hrobce
Kryry
Řehlovice
Prackovice nad Labem
Litoměřice horní nádraží
Ústí nad Labem-Střekov
Libochovice
Želina
Žihle
Chyše
Kadaňský Rohozec
Štědrá
Toužim
Zbečno
Osek
Klobuky v Čechách
Světec
Moldava v Krušných horách
Dalovice
Nové Strašecí
Žalhostice
Louka u Litvínova
Slaný
Krásný Jez
Karlovy Vary-Březová
Hájek
Ka

c:\Users\pazera\anaconda3\envs\starter\Lib\site-packages\networkx\readwrite\json_graph\node_link.py:287: FutureWarning: 
The default value will be changed to `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_graph(data, edges="links") to preserve current behavior, or
  nx.node_link_graph(data, edges="edges") for forward compatibility.
  warnings.warn(


# Deletion of nodes Czechia

In [ ]:
from networkx import NetworkXError
stations_to_remove  = [
    "Anděl", "Bořislavka", "Budějovická", "Černý Most", "Českomoravská", "Chodov", "Dejvická", "Depo Hostivař", "Flora", "Florenc",
    "Háje", "Hlavní nádraží", "Hloubětín", "Hradčanská", "Hůrka", "I. P. Pavlova", "Invalidovna", "Jinonice", "Jiřího z Poděbrad", "Kačerov",
    "Karlovo náměstí", "Kobylisy", "Kolbenova", "Křižíkova", "Ládví", "Letňany", "Luka", "Lužiny", "Malostranská", "Můstek", "Muzeum",
    "Nádraží Holešovice", "Nádraží Veleslavín", "Náměstí Míru", "Náměstí Republiky", "Národní třída", "Nemocnice Motol", "Nové Butovice", "Opatov",
    "Palmovka", "Pankrác", "Petřiny", "Pražského povstání", "Prosek", "Radlická", "Rajská zahrada", "Roztyly", "Skalka", "Smíchovské nádraží",
    "Staroměstská", "Stodůlky", "Strašnická", "Střížkov", "Vltavská", "Vyšehrad", "Vysočanská", "Zličín", "Želivského", "Nová Ves u Kunčiny MPD"
]
for node in stations_to_remove:
    try:
        G.remove_node(node)
    except (NetworkXError, KeyError):
        print(node)

Českomoravská
Pankrác


# Addition of edges Czechia

In [ ]:
from geopy.distance import geodesic
edges_list = [("Praha hlavní nádraží", "Praha-Libeň"), ("Praha-Vysočany", "Praha hlavní nádraží"), 
              ("Praha hlavní nádraží", "Praha-Vršovice"), ("Praha hlavní nádraží", "Praha-Smíchov"), 
              ("Praha-Vršovice", "Praha-Zahradní Město"), ("Praha-Zahradní Město", "Praha-Hostivař"),
              ("Praha-Běchovice", "Praha-Libeň"), ("Praha-Běchovice", "Úvaly"), ("Praha-Horní Počernice", "Mstětice"),
              ("Kostomlaty nad Labem", "Lysá nad Labem"), ("Lysá nad Labem", "Stará Boleslav"), 
              ("Praha-Satalice", "Praha-Čakovice"),("Praha-Satalice", "Praha-Vysočany"), ("Praha-Satalice", "Praha-Horní Počernice"),
               ("Praha-Holešovice", "Roztoky u Prahy"), ("Praha-Holešovice", "Praha-Vysočany"), ("Praha-Holešovice", "Praha hlavní nádraží"),
            ("Praha-Dejvice", "Praha-Veleslavín"), ("Praha-Veleslavín", "Praha-Ruzyně"), ("Praha-Krč", "Praha-Zahradní Město"),
               ("Praha-Krč", "Praha-Vršovice"), ("Praha-Žvahov", "Praha-Řeporyje"), ("Praha-Zahradní Město", "Praha-Libeň"), 
               ("Český Brod", "Poříčany"), ("Poříčany", "Pečky"), ("Pečky", "Plaňany"), ("Bečváry", "Ratboř"), 
               ("Kolín", "Velim"), ("Ratboř", "Kolín"),("Kolín", "Kutná Hora město"), ("Kolín", "Záboří nad Labem"),
                 ("Záboří nad Labem", "Řečany nad Labem"),
              ("Řečany nad Labem", "Přelouč"), ("Přelouč", "Pardubice hlavní nádraží"), 
              ("Kostěnice", "Pardubice hlavní nádraží"), ("Pardubice hlavní nádraží", "Medlešice"), ("Chrudim město", "Chrudim"),
              ("Hrochův Týnec", "Moravany"), ("Moravany", "Uhersko"), ("Uhersko", "Zámrsk"), ("Zámrsk", "Choceň"),
              ("Cerekvice nad Loučnou", "Litomyšl"), ("Dlouhá Třebová", "Česká Třebová"), 
              ("Rudoltice v Čechách", "Zábřeh na Moravě"), ("Moravičany", "Červenka"), ("Červenka", "Štěpánov"),
            ("Štěpánov", "Olomouc hlavní nádraží"), ("Velká Bystřice", "Hlubočky-Mariánské Údolí"), 
            ("Hlubočky-Mariánské Údolí", "Hlubočky"), ("Hrubá Voda", "Domašov nad Bystřicí"), ("Domašov nad Bystřicí", "Moravský Beroun"),
            ("Moravský Beroun", "Dětřichov nad Bystřicí"), ("Dětřichov nad Bystřicí", "Valšov"), ("Valšov", "Bruntál"),
            ("Bruntál", "Milotice nad Opavou"), ("Kunov", "Milotice nad Opavou"),
            ("Milotice nad Opavou", "Brantice"), ("Brantice", "Krnov"), ("Mikulovice", "Jindřichov ve Slezsku"),
              ("Černý Kříž", "Stožec"), ("Černý Kříž", "Volary"), ("Černá v Pošumaví", "Horní Planá"), 
              ("Černá v Pošumaví", "Polečnice"), ("Hořice na Šumavě", "Kájov"), ("Kájov", "Český Krumlov"),
              ("Rybník", "Omlenice"), ("Omlenice", "Kaplice"), ("Kaplice", "Velešín"), 
              ("Velešín", "Holkov"), ( "Holkov", "Kamenný Újezd u Českých Budějovic"),
              ("Kamenný Újezd u Českých Budějovic", "Včelná"), ("Nová Ves u Českých Budějovic", "Borovany"), 
              ("Borovany", "Jílovice"), ("Jílovice", "Nové Hrady"), ("Nové Hrady", "České Velenice"), 
              ("Jarošov nad Nežárkou", "Popelín"), ("Volary", "Zbytiny"), ("Chotýčany", "Ševětín"), ("Písek město", "Záhoří"),
               ("Červená nad Vltavou", "Branice"), ("Branice", "Milevsko"), ("Balkova Lhota", "Tábor"),
               ("Chotýčany", "Ševětín"), ("Praha-Radotín", "Dobřichovice"), ("Liteň", "Zadní Třebaň"), 
               ("Vrané nad Vltavou", "Praha-Zbraslav"), ("Beroun", "Karlštejn"),("Nučice", "Rudná u Prahy"), 
               ("Jeneč", "Rudná u Prahy"), ("Praha-Zličín", "Praha-Stodůlky"), ("Praha-Stodůlky", "Praha-Žvahov"),
               ("Praha-Smíchov", "Praha-Vršovice"), ("Podlešín", "Slaný"), ("Peruc", "Klobuky v Čechách"),
               ("Louny", "Libochovice"),("Ohníč", "Světec"), ("Úštěk", "Blíževedly"), ("Dolní Podluží", "Rybniště"), 
               ("Jablonec nad Nisou dolní nádraží", "Vesec u Liberce"), ("Horka u Staré Paky", "Mostek"), 
               ("Martinice v Krkonoších", "Kunčice nad Labem"), ("Trutnov hlavní nádraží", "Trutnov střed"),
               ("Police nad Metují", "Hronov"), ("Svitavy", "Třebovice v Čechách"),
            ("Vlkaneč", "Golčův Jeníkov"), ("Golčův Jeníkov", "Čáslav"),  ("Žďár nad Sázavou", "Nové Město na Moravě"), 
              ("Okrouhlice", "Havlíčkův Brod"), ("Lískovec u Frýdku", "Frýdek-Místek"), ("Frýdek-Místek", "Baška"), 
              ("Frýdlant nad Ostravicí", "Kunčice pod Ondřejníkem"), ("Veselí nad Moravou", "Strážnice"), 
              ("Sedlec u Mikulova", "Mikulov na Moravě"), ("Chýnov", "Obrataň"), ("Obrataň", "Pacov"),
               ("Pacov", "Nová Cerekev"), ("Obrataň", "Křeč"), ("Křeč", "Černovice u Tábora"), ("Černovice u Tábora", "Chválkov"),
               ("Chválkov", "Včelnička"), ( "Včelnička", "Kamenice nad Lipou"), ("Kamenice nad Lipou", "Nová Včelnice"), 
               ("Nová Včelnice", "Lovětín"), ("Lovětín", "Horní Skrýchov"), ("Jeneč", "Středokluky"), ("Kralupy nad Vltavou předměstí", "Velvary"), 
               ("Nelahozeves", "Kralupy nad Vltavou"), ("Nové Sedlo u Lokte", "Karlovy Vary"), ("Karlovy Vary dolní nádraží", "Karlovy Vary-Březová"),
               ("Františkovy Lázně", "Vojtanov"), ("Kdyně", "Dobříkov na Šumavě"), ("Čkyně", "Bohumilice v Čechách"), 
               ("Vodňany", "Číčenice"), ("Kralice nad Oslavou", "Rapotice"), ("Zastávka u Brna", "Střelice"), 
               ("Troubsko", "Brno-Horní Heršpice"), ("Bojkovice", "Slavičín"), ("Karolinka", "Velké Karlovice"),
                ("Bečváry", "Hatě"), ("Kutná Hora město","Malešov"), ("Roztoky u Křivoklátu", "Městečko u Křivoklátu"), 
              ("Lašovice",  "Rakovník"), ("Horní Bříza", "Kaznějov"), ("Blatno u Jesenice", "Jesenice"),
              ("Blatno u Jesenice", "Petrohrad"), ("Blatno u Jesenice", "Lubenec"), ("Lubenec", "Chyše"),
              ("Měcholupy", "Žatec"), ("Dolejší Hůrky", "Lišany u Žatce"), ("Lišany u Žatce", "Postoloprty"), 
              ("Louny", "Lenešice"), ("Hřivice", "Louny předměstí")
              ]

for u, v in edges_list:
    G.add_edge(u, v)
    point_u = G.nodes[u]['geometry']
    point_v = G.nodes[v]['geometry']
    coords_u = point_u.coords[0]
    coords_v = point_v.coords[0]
    latlon_u = (coords_u[1], coords_u[0])
    latlon_v = (coords_v[1], coords_v[0])
    total_distance = geodesic(latlon_u, latlon_v).meters
    G[u][v]['distance'] = total_distance   

# Deletion of edges

In [29]:
edges_to_delete = [("Praha-Horní Počernice", "Úvaly"), ("Moravičany", "Mladeč"), ("Mladeč", "Litovel"), ("Štěpánov", "Bohuňovice"),
("Hodkovice nad Mohelkou", "Jeřmanice")] 
for edge in edges_to_delete:
    try:
        G.remove_edge(edge[0], edge[1])
    except (NetworkXError, KeyError):
        print(edge)

# Saving the fixed graph

In [ ]:
graph_to_geojson(G=G, path="../graphs/cze-railroad-network.json")

c:\Users\pazera\anaconda3\envs\starter\Lib\site-packages\networkx\readwrite\json_graph\node_link.py:142: FutureWarning: 
The default value will be `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_data(G, edges="links") to preserve current behavior, or
  nx.node_link_data(G, edges="edges") for forward compatibility.
  warnings.warn(


# Map of both graphs

In [ ]:
m = folium.Map(location=[48, 17], zoom_start=7)
G = graph_from_geojson(path=f"../graphs/svk-railroad-network.json")
G2 = graph_from_geojson(path=f"../graphs/cze-railroad-network.json")
for node1, node2 in G.edges():
    if node1 in G.nodes and node2 in G.nodes and 'geometry' in G.nodes[node1] and 'geometry' in G.nodes[node2]:
        geom1 = G.nodes[node1]['geometry']
        geom2 = G.nodes[node2]['geometry']
        if isinstance(geom1, Point) and isinstance(geom2, Point):
            lat1, lon1 = geom1.y, geom1.x
            lat2, lon2 = geom2.y, geom2.x
            folium.PolyLine([[lat1, lon1], [lat2, lon2]], color='blue', weight=1).add_to(m)
        else:
            print(f"Warning: Skipping edge between {node1} and {node2} due to non-Point geometry.")
    else:
        print(f"Warning: Skipping edge between {node1} and {node2} because one or both nodes are missing geometry information.")

for node1, node2 in G2.edges():
    if node1 in G2.nodes and node2 in G2.nodes and 'geometry' in G2.nodes[node1] and 'geometry' in G2.nodes[node2]:
        geom1 = G2.nodes[node1]['geometry']
        geom2 = G2.nodes[node2]['geometry']
        if isinstance(geom1, Point) and isinstance(geom2, Point):
            lat1, lon1 = geom1.y, geom1.x
            lat2, lon2 = geom2.y, geom2.x
            folium.PolyLine([[lat1, lon1], [lat2, lon2]], color='blue', weight=1).add_to(m)
        else:
            print(f"Warning: Skipping edge between {node1} and {node2} due to non-Point geometry.")
    else:
        print(f"Warning: Skipping edge between {node1} and {node2} because one or both nodes are missing geometry information.")
# Mark the nodes (stations) with red dots
for node, data in G.nodes(data=True):
    if 'geometry' in data and isinstance(data['geometry'], Point):
        folium.CircleMarker(
            location=[data['geometry'].y, data['geometry'].x],
            radius=1,
            color='red',
            fill=True,
            fill_color='red',
            fill_opacity=0.7
        ).add_to(m)

for node, data in G2.nodes(data=True):
    if 'geometry' in data and isinstance(data['geometry'], Point):
        folium.CircleMarker(
            location=[data['geometry'].y, data['geometry'].x],
            radius=1,
            color='red',
            fill=True,
            fill_color='red',
            fill_opacity=0.7
        ).add_to(m)
# Save or display the map
m.save(f'visualizations/combined_railroads.html')